In [115]:
import numpy as np
import math
def jma(close:np.array, length:int=7, phase:float=0):
    """
    Jurik Moving Average implementation from: https://c.mql5.com/forextsd/forum/164/jurik_1.pdf
    Discussion thread and various implementations: https://www.mql5.com/en/forum/179011
    """
    jma = np.zeros_like(close)
    Volty = np.zeros_like(close)
    vSum = np.zeros_like(close)
    Kv = det0 = det1 = ma2 = 0.0
    jma[0] = ma1 = uBand = lBand = close[0]
    # Static variables
    SumLen = 10
    len = 0.5*(length-1)
    PR = 0.5 if phase<-100 else 2.5 if phase>100 else phase*0.01+1.5
    len1 = max((math.log(math.sqrt(len))/math.log(2.0))+2.0, 0)
    pow1 = max(len1-2.0, 0.5)
    len2 = math.sqrt(len)*len1
    bet = len2/(len2+1)
    beta = 0.45*(length-1)/(0.45*(length-1)+2.0)
    
    for i in range(1, close.shape[0]):
        price = close[i]
        # Price volatility
        del1 = price-uBand
        del2 = price-lBand
        Volty[i] = max(abs(del1),abs(del2)) if abs(del1)!=abs(del2) else 0
        # Relative price volatility factor
        vSum[i] = vSum[i-1] + (Volty[i]-Volty[max(i-SumLen,0)])/SumLen
        avgVolty = np.average(vSum[max(i-65,0):i+1])
        dVolty = 0 if avgVolty==0 else Volty[i]/avgVolty
        rVolty = max(1.0, min(math.pow(len1, 1/pow1), dVolty))
        # Jurik volatility bands
        pow2 = math.pow(rVolty, pow1)
        Kv = math.pow(bet, math.sqrt(pow2))
        uBand = price if (del1 > 0) else price - (Kv*del1)
        lBand = price if (del2 < 0) else price - (Kv*del2)
        # Jurik Dynamic Factor
        power = math.pow(rVolty, pow1)
        alpha = math.pow(beta, power)
        # 1st stage - prelimimary smoothing by adaptive EMA
        ma1 = ((1-alpha)*price)+(alpha*ma1) #
        # 2nd stage - one more prelimimary smoothing by Kalman filter
        det0 = ((price-ma1)*(1-beta))+(beta*det0)
        ma2 = ma1+PR*det0
        # 3rd stage - final smoothing by unique Jurik adaptive filter
        det1 = ((ma2-jma[i-1])*(1-alpha)*(1-alpha))+(alpha*alpha*det1)
        jma[i] = jma[i-1] + det1
    return jma


In [116]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

dfin = pd.read_csv (r"JMA-test.csv")
df=pd.DataFrame()

diff=0
tests=['Spike', 'Impulse', 'Triangle', 'Sawtooth', 'Sine', 'Chirp', 'White', 'Gauss', 'B', 'HF', 'Impulse+HF', 'Sawtooth+HF', 'Sine+G', 'Chirp+G', 'Market', 'Complex']
for test in tests:
    df['input']=dfin[test]
    df['control']=dfin[test+'-JMA']
    df['jma']=jma(df['input'], length=10, phase=0)

    fig = go.Figure(data=go.Scatter(y=df['input'], name='input'))
    fig.add_scatter(y=df['control'], name='control', marker_color='yellow')
    fig.add_scatter(y=df['jma'], name='jma', marker_color='white')
    fig.update_layout(xaxis_rangeslider_visible=False, margin=dict(l=0, r=10, t=0, b=0), showlegend=False, title={'text':test, 'y':0.9})
    fig.layout.template = 'plotly_dark'
    fig.show()    

